In [39]:
from tsfresh.examples import load_robot_execution_failures
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

from sklearn.metrics import f1_score

### force and torque measurements on a robot after failure detection

In [4]:
download_robot_execution_failures()
df_ts, y = load_robot_execution_failures()

In [8]:
df_ts.columns

Index(['id', 'time', 'F_x', 'F_y', 'F_z', 'T_x', 'T_y', 'T_z'], dtype='object')

In [12]:
df_ts.head()

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0


In [13]:
# check the frequency of the two classes
print(f"{y.mean():.2f}")

0.24


### features

In [23]:
# extract time-series features, impute missing values
extracted_features = impute(extract_features(df_ts, column_id="id", column_sort="time"))
# select features based on relevance to the target
features_filtered = select_features(extracted_features, y)

# impute
# https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#:~:text=tsfresh.utilities.dataframe_functions%20module%20%C2%B6%20Utility%20functions%20for%20handling%20the,to%20handle%20NaN%20and%20inf%20in%20the%20DataFrames.

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:25<00:00,  1.27s/it]
C:\Users\stakk\miniconda3\envs\timeseries\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['F_x__partial_autocorrelation__lag_7'
 'F_x__partial_autocorrelation__lag_8'
 'F_x__partial_autocorrelation__lag_9' ...
 'T_z__matrix_profile__feature_"median"__threshold_0.98'
 'T_z__matrix_profile__feature_"25"__threshold_0.98'
 'T_z__matrix_profile__feature_"75"__threshold_0.98'] did not have any finite values. Filling with zeros.
  warnings.warn(


### find good values for the number of neighbors by doing a grid search

In [26]:
%%time

param_search = {
    'metric' : ['dtw'],
    'n_neighbors': [1, 2, 3]
}
gsearch = GridSearchCV(
    estimator=KNeighborsTimeSeriesClassifier(),
    cv=TimeSeriesSplit(n_splits=2),
    param_grid=param_search
)
gsearch.fit(
    features_filtered,
    y
)

CPU times: total: 1min 40s
Wall time: 1min 44s


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=KNeighborsTimeSeriesClassifier(),
             param_grid={'metric': ['dtw'], 'n_neighbors': [1, 2, 3]})

### check metrics

In [66]:
%%time

gsearch.best_estimator_.predict(features_filtered.iloc[])

ValueError: Expected 2D array, got 1D array instead:
array=[ 7.00000000e+00  2.50000000e+01  1.29099445e+00  5.00000000e+00
  1.57142857e+00  1.30000000e+01  7.60000000e+01  2.25092574e+00
  3.00000000e+00  4.22222222e+00  2.05480467e+00  4.00000000e+00
  1.00000000e+01  6.24117752e-01  3.00000000e+00  6.02026137e+00
  2.44948974e+00  9.00000000e+01  3.28571429e+00  4.33384612e+00
  1.87822222e+01  1.98400000e+01  9.15555556e-01  9.56846673e-01
  2.00000000e-01  3.42279937e+00  1.17155556e+01  5.13851685e+00
  8.68063668e+00  5.72424583e+00 -1.00208044e-01 -1.00208044e-01
  8.00000000e-01  6.28571429e+00  6.14809121e+00  6.00000000e-01
  5.33333333e-01  5.30000000e+01  1.00000000e+01  5.71428571e-01
  6.17795956e+00  2.26948639e+00  3.63000000e+02  4.91934955e+00
  3.25000000e+00  2.58233614e+00  1.22681886e+01  7.21110255e+00
  1.53268913e+01  1.30000000e+01  8.10000000e-01  5.71428571e-01
  4.00000000e+00  5.16397779e-01  4.66666667e-01  2.14993540e+00
  4.62222222e+00  1.20100000e+0

In [36]:
f1_score(y, y_pred)

(88, 682)

In [44]:
features_filtered.shape

(88, 682)

In [53]:
features_filtered.iloc[0:2,:]

,F_x__value_count__value_-1,F_x__abs_energy,F_x__root_mean_square,T_y__absolute_maximum,F_x__mean_n_absolute_max__number_of_maxima_7,F_x__range_count__max_1__min_-1,F_y__abs_energy,F_y__root_mean_square,F_y__mean_n_absolute_max__number_of_maxima_7,T_y__variance,T_y__standard_deviation,F_y__absolute_maximum,T_x__absolute_maximum,"F_x__fft_coefficient__attr_""abs""__coeff_1",F_x__absolute_maximum,"T_y__fft_coefficient__attr_""abs""__coeff_1",T_y__root_mean_square,T_y__abs_energy,T_y__mean_n_absolute_max__number_of_maxima_7,F_z__standard_deviation,F_z__variance,"F_z__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""",F_x__variance,F_x__standard_deviation,F_x__ratio_value_number_to_time_series_length,T_x__standard_deviation,T_x__variance,"T_x__fft_coefficient__attr_""abs""__coeff_1","T_y__fft_coefficient__attr_""abs""__coeff_2",F_x__cid_ce__normalize_True,F_x__partial_autocorrelation__lag_1,F_x__autocorrelation__lag_1,T_y__percentage_of_reoccurring_datapoints_to_all_datapoints,T_x__mean_n_absolute_max__number_of_maxima_7,"T_y__fft_coefficient__attr_""abs""__coeff_4",T_x__ratio_value_number_to_time_series_length,F_x__lempel_ziv_complexity__bins_100,"F_z__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","F_y__fft_coefficient__attr_""abs""__coeff_0",T_y__percentage_of_reoccurring_values_to_all_values,"T_y__fft_coefficient__attr_""abs""__coeff_3","F_x__fft_coefficient__attr_""abs""__coeff_2",T_x__abs_energy,T_x__root_mean_square,"T_y__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","F_y__fft_coefficient__attr_""abs""__coeff_1","F_z__fft_coefficient__attr_""abs""__coeff_1","T_x__fft_coefficient__attr_""abs""__coeff_5","T_x__fft_coefficient__attr_""abs""__coeff_2","F_x__fft_coefficient__attr_""abs""__coeff_0","F_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""",T_z__mean_n_absolute_max__number_of_maxima_7,T_z__abs_energy,T_z__root_mean_square,T_y__ratio_value_number_to_time_series_length,F_y__standard_deviation,F_y__variance,"T_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""",T_z__variance,T_z__standard_deviation,F_x__variance_larger_than_standard_deviation,"F_x__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","F_x__fft_coefficient__attr_""abs""__coeff_4",F_z__minimum,T_z__range_count__max_1__min_-1,"T_z__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","T_x__fft_coefficient__attr_""abs""__coeff_4","F_x__fft_coefficient__attr_""abs""__coeff_3","T_y__linear_trend__attr_""stderr""",T_z__absolute_maximum,F_x__percentage_of_reoccurring_datapoints_to_all_datapoints,"F_z__fft_coefficient__attr_""abs""__coeff_3","F_z__fft_coefficient__attr_""abs""__coeff_6",F_y__ratio_value_number_to_time_series_length,"F_z__fft_aggregated__aggtype_""centroid""",T_z__ratio_value_number_to_time_series_length,"F_x__fft_aggregated__aggtype_""variance""","F_z__fft_coefficient__attr_""abs""__coeff_2",T_x__percentage_of_reoccurring_datapoints_to_all_datapoints,F_y__range_count__max_1__min_-1,"T_z__fft_coefficient__attr_""abs""__coeff_1","F_x__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""",...,"T_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""",F_y__quantile__q_0.8,F_y__number_crossing_m__m_1,F_x__number_crossing_m__m_1,"F_x__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8",F_x__energy_ratio_by_chunks__num_segments_10__segment_focus_3,T_y__has_duplicate_max,"T_z__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""var""",T_y__ratio_beyond_r_sigma__r_2,T_x__energy_ratio_by_chunks__num_segments_10__segment_focus_4,"T_z__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""",F_x__count_above_mean,"T_x__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6",T_y__minimum,T_z__maximum,T_x__lempel_ziv_complexity__bins_2,F_z__autocorrelation__lag_7,F_x__fourier_entropy__bins_2,"T_z__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",T_y__kurtosis,"F_x__change_quantiles_